In [5]:
import os
import random
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import yaml
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import DataLoader, Dataset
from retrieval_models import ModernNCA
from transformers import AutoTokenizer, AutoModel

In [6]:
# Define a custom dataset to keep track of indices
class IndexedTensorDataset(Dataset):
    def __init__(self, tensors_num, tensors_cat, targets):
        self.tensors_num = tensors_num
        self.tensors_cat = tensors_cat
        self.targets = targets
        self.indices = torch.arange(len(tensors_num), dtype=torch.long)

    def __getitem__(self, index):
        return (self.tensors_num[index], self.tensors_cat[index], self.targets[index], self.indices[index])

    def __len__(self):
        return len(self.tensors_num)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def load_config(config_file):
    with open(config_file, 'r') as file:
        config = yaml.safe_load(file)
    return config

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def label_encode(X, categorical_columns):
    le_categorical = {}
    for col in categorical_columns:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
        le_categorical[col] = le  # Save each encoder if needed for inverse transformation later
    return X, le_categorical

def llm_encoder(X, categorical_columns):
    embeddings = []
    for idx in range(X.shape[0]):
        string = ''
        for column in categorical_columns:
            string += f"{column}: {X[column][idx]}. "
        embeddings.append(string)
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-MiniLM-L6-v2')
    model = AutoModel.from_pretrained('sentence-transformers/paraphrase-MiniLM-L6-v2')
    encoded_input = tokenizer(embeddings, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        embeddings = model(**encoded_input)
    embeddings = mean_pooling(embeddings, encoded_input['attention_mask'])
    embeddings = pd.DataFrame(embeddings.numpy(), columns=[f'embedding_{i}' for i in range(embeddings.shape[1])])
    X = pd.concat([X, embeddings], axis=1)
    X = X.drop(columns=categorical_columns)
    return X

def onehot_encode(X, categorical_columns):
    X = pd.get_dummies(X, columns=categorical_columns, drop_first = True)
    return X

def load_dataset(config):
    # Fetch the dataset
    data = fetch_openml(config['dataset']['name'], version=config['dataset']['version'], as_frame=True)
    X = data['data'].copy()
    y = data['target']

    # Downsample the dataset
    X = X[:config['dataset']['n_samples']]
    y = y[:config['dataset']['n_samples']]

    # Identify categorical and numerical columns
    categorical_columns = X.select_dtypes(include=['category', 'object']).columns.tolist()
    numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

    if config['dataset']['cat_encode'] == 'label':
        X, _ = label_encode(X, categorical_columns)
    elif config['dataset']['cat_encode'] == 'onehot':
        X = onehot_encode(X, categorical_columns)
    elif config['dataset']['cat_encode'] == 'llm':
        X = llm_encoder(X, categorical_columns)
    
    # Scale numerical columns
    numerical_transformer = StandardScaler()
    X[numerical_columns] = numerical_transformer.fit_transform(X[numerical_columns])

    # Encode the target variable
    le_target = LabelEncoder()
    y = le_target.fit_transform(y)

    # Convert X and y to tensor
    y = torch.tensor(y, dtype=torch.long)
    if config['dataset']['all_num']:
        X_num = torch.tensor(X.values, dtype=torch.float32)
        X_cat = X_num.clone()
        d_in_num = X_num.shape[1]
        d_in_cat = 0
    else:
        X_num = torch.tensor(X[numerical_columns].values, dtype=torch.float32)
        X_cat = torch.tensor(X.drop(columns=numerical_columns).values, dtype=torch.float32)
        d_in_num = X_num.shape[1]
        d_in_cat = X_cat.shape[1]

    return (X_num, X_cat, y, d_in_num, d_in_cat, le_target)


def evaluate(config, model, test_loader, criterion, device, X_num_train, X_cat_train, y_train):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0.0

    with torch.no_grad():
        for X_num_batch, X_cat_batch, y_batch, idx_batch in test_loader:
            X_num_batch = X_num_batch.to(device)
            X_cat_batch = X_cat_batch.to(device) if not config['dataset']['all_num'] else None
            y_batch = y_batch.to(device)

            # Use entire training data as candidates during evaluation
            candidate_x_num = X_num_train.to(device) 
            candidate_x_cat = X_cat_train.to(device) if not config['dataset']['all_num'] else None
            candidate_y = y_train.to(device)

            # Forward pass with separate categorical and numerical features
            logits = model(
                x_num=X_num_batch,
                x_cat=X_cat_batch,
                y=None,
                candidate_x_num=candidate_x_num,
                candidate_x_cat=candidate_x_cat,
                candidate_y=candidate_y,
                is_train=False
            )

            # Convert logits to predictions
            _, predicted = torch.max(logits, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

            # Compute loss
            loss = criterion(logits, y_batch)
            test_loss += loss.item() * y_batch.size(0)

    accuracy = 100 * correct / total
    test_loss = test_loss / total

    print(f'Test Loss: {test_loss:.4f} | Test Accuracy: {accuracy:.2f}%')

def train(config):
    set_seed(config['dataset']['random_state'])

    X_num, X_cat, y, d_in_num, d_in_cat, le_target = load_dataset(config)
    output_classes = len(le_target.classes_)

    X_num_train, X_num_test, X_cat_train, X_cat_test, y_train, y_test = train_test_split(
        X_num, X_cat, y, test_size=config['dataset']['test_size'], random_state=config['dataset']['random_state']
    )

    # Use IndexedTensorDataset for separate numerical and categorical data
    train_dataset = IndexedTensorDataset(X_num_train, X_cat_train, y_train)
    test_dataset = IndexedTensorDataset(X_num_test, X_cat_test, y_test)

    train_loader = DataLoader(
        train_dataset, batch_size=config['dataset']['batch_size'], shuffle=True,
        pin_memory=True, num_workers=4
    )
    test_loader = DataLoader(
        test_dataset, batch_size=config['dataset']['batch_size'], shuffle=False,
        pin_memory=True, num_workers=4
    )

    # Initialize the ModernNCA model
    model = ModernNCA(
        d_in_num=d_in_num,
        d_in_cat=d_in_cat,
        d_out=output_classes,
        dim=config['model']['dim'],
        dropout=config['model']['dropout'],
        n_frequencies=config['model']['n_frequencies'],
        frequency_scale=config['model']['frequency_scale'],
        d_embedding=config['model']['d_embedding'],
        lite=config['model']['lite'],
        temperature=config['model']['temperature'],
        sample_rate=config['model']['sample_rate'],
        use_llama=config['model']['use_llama'],
        llama_model_name=config['model']['llama_model_name'],
        start_layer=config['model']['start_layer'],
        end_layer=config['model']['end_layer']
    )

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=config['training']['learning_rate'],
        weight_decay=config['training']['weight_decay']
    )

    # Training loop
    for epoch in range(config['training']['epochs']):
        model.train()
        start_time = time.time()
        epoch_loss = 0.0
        correct = 0
        total = 0

        for itr, (X_num_batch, X_cat_batch, y_batch, idx_batch) in enumerate(train_loader):
            X_num_batch = X_num_batch.to(device)
            X_cat_batch = X_cat_batch.to(device) if not config['dataset']['all_num'] else None
            y_batch = y_batch.to(device)

            # Exclude current batch for candidates
            mask = torch.isin(torch.arange(X_num_train.shape[0]), idx_batch)
            candidate_x_num = X_num_train[~mask].to(device)
            candidate_x_cat = X_cat_train[~mask].to(device) if not config['dataset']['all_num'] else None
            candidate_y = y_train[~mask].to(device)

            optimizer.zero_grad()

            # Forward pass with separate categorical and numerical features
            logits = model(
                x_num=X_num_batch,
                x_cat=X_cat_batch,
                y=y_batch,
                candidate_x_num=candidate_x_num,
                candidate_x_cat=candidate_x_cat,
                candidate_y=candidate_y,
                is_train=True
            )
            loss = criterion(logits, y_batch)

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * y_batch.size(0)
            _, predicted = torch.max(logits, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

            if itr % 50 == 0:
                print(f'Epoch [{epoch+1}/{config["training"]["epochs"]}]: Batch [{itr+1}/{len(train_loader)}] | Accuracy: {correct/total:.2f}')

        epoch_loss = epoch_loss / total
        epoch_acc = 100 * correct / total
        epoch_time = time.time() - start_time

        print(f'Epoch [{epoch+1}/{config["training"]["epochs"]}] | Loss: {epoch_loss:.4f} | '
                f'Accuracy: {epoch_acc:.2f}% | Time: {epoch_time:.2f}s')

    evaluate(config, model, test_loader, criterion, device, X_num_train, X_cat_train, y_train)

    # save the model
    path = os.getcwd() + '/TabLLM/initial_exp/' + config['training']['model_path']
    torch.save(model.state_dict(), path)
    model.load_state_dict(torch.load(path))

    evaluate(config, model, test_loader, criterion, device, X_num_train, X_cat_train, y_train)

In [7]:
config = load_config(os.getcwd() + '/TabLLM/initial_exp/retrieval_config.yaml')

In [8]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
train(config)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Epoch [1/10]: Batch [1/250] | Accuracy: 0.81
Epoch [1/10]: Batch [51/250] | Accuracy: 0.83
Epoch [1/10]: Batch [101/250] | Accuracy: 0.84
Epoch [1/10]: Batch [151/250] | Accuracy: 0.84
Epoch [1/10]: Batch [201/250] | Accuracy: 0.84
Epoch [1/10] | Loss: 0.3278 | Accuracy: 84.36% | Time: 54.68s
Epoch [2/10]: Batch [1/250] | Accuracy: 0.84
Epoch [2/10]: Batch [51/250] | Accuracy: 0.86
Epoch [2/10]: Batch [101/250] | Accuracy: 0.86
Epoch [2/10]: Batch [151/250] | Accuracy: 0.85
Epoch [2/10]: Batch [201/250] | Accuracy: 0.86
Epoch [2/10] | Loss: 0.3104 | Accuracy: 85.54% | Time: 54.19s
Epoch [3/10]: Batch [1/250] | Accuracy: 0.69
Epoch [3/10]: Batch [51/250] | Accuracy: 0.86
Epoch [3/10]: Batch [101/250] | Accuracy: 0.85
Epoch [3/10]: Batch [151/250] | Accuracy: 0.85
Epoch [3/10]: Batch [201/250] | Accuracy: 0.85
Epoch [3/10] | Loss: 0.3076 | Accuracy: 85.49% | Time: 54.35s
Epoch [4/10]: Batch [1/250] | Accuracy: 0.88
Epoch [4/10]: Batch [51/250] | Accuracy: 0.86
Epoch [4/10]: Batch [101/25